# Helix demo using Python scripting and `roslibpy`
This notebook contains instructions and step by step examples on how to use functions associated to Helix v1 and troubleshooting in case of failure. This notebook should be executed one cell at a time while checking that outputs are coherent.
## Setup
### Connect to Pi's rosbridge server

In [1]:
import roslibpy
import time
import numpy as np

# Connect to the Pi's rosbridge server over port 9090
client = roslibpy.Ros(host='localhost', port=9090) # The host is the Pi's IP address
client.run()

# Check that the connecting worked
print('Is ROS connected?', client.is_connected)

Is ROS connected? True


If the output is: `Is ROS connected ? True`, proceed to the next step. If not, check the host and port addresses, and make sure that all docker containers on the RPi are running correctly (check instructions for launching the containers [here](https://github.com/helix-robotics-ag/ros-helix/tree/main))

### Check available topics and services

Verify that all desired topics and services are available - in particular, there should be a range of topics and services under the namespace `/tendon_transmission_node`

In [2]:
# Check which topics can be seen
print('ROS Topics: ', client.get_topics())

# Check which services can be seen
print('ROS Services: ', client.get_services())

ROS Topics:  ['/clicked_point', '/client_count', '/connected_clients', '/diagnostics', '/gripper_joint_position_controller/commands', '/gripper_joint_position_controller/transition_event', '/gripper_joint_state_broadcaster/dynamic_joint_states', '/gripper_joint_state_broadcaster/joint_states', '/gripper_joint_state_broadcaster/transition_event', '/helix_cartesian_control_node/cartesian_state', '/helix_cartesian_control_node/delta_increment', '/helix_cartesian_control_node/dxdyl_command', '/helix_cartesian_control_node/dxdyl_state', '/helix_gripper_node/command_increment', '/initialpose', '/joint_states', '/motor_head_joint_effort_controller/commands', '/motor_head_joint_effort_controller/transition_event', '/motor_head_joint_position_controller/commands', '/motor_head_joint_position_controller/transition_event', '/motor_head_joint_state_broadcaster/dynamic_joint_states', '/motor_head_joint_state_broadcaster/joint_states', '/motor_head_joint_state_broadcaster/transition_event', '/move_b

## Calibration

For now the calibration is done through different functions depending on the state of the manipulator. The workflow will be roughly as follows:
1. Loosen up all tendons and make the arm roughly straight
2. Add tension to the tendons and straighten up the manipulator to it's zero state
3. Create or update existing calibration file

### Loosen up all tendons

This step is required if some tendons are tense and the arm isn't straight. First, the controller needs to be set to current control mode.

In [ ]:
# Switch to current control
srv = roslibpy.Service(client, '/tendon_transmission_node/switch_to_current_control', 'std_srvs/Trigger')
req = roslibpy.ServiceRequest()
result = srv.call(req)
print(result)

Then use the following service to release the tension: 

In [ ]:
# Unwind tendon
srv = roslibpy.Service(client, '/tendon_transmission_node/set_current', 'helix_transmission_interfaces/SetCurrent')
req = roslibpy.ServiceRequest({"current" : 3.0}) # 3mA is the smallest current that can be applied. A positive value loosens the tendons.
result = srv.call(req)
print(result)

Stop the motors when tendons are no longer tense:

In [ ]:
# Stop motors from turning
srv = roslibpy.Service(client, '/tendon_transmission_node/set_current', 'helix_transmission_interfaces/SetCurrent')
req = roslibpy.ServiceRequest({"current" : 0.0})
result = srv.call(req)
print(result)

### Add tension to the arm

In [ ]:
# Add tension
srv = roslibpy.Service(client, '/tendon_transmission_node/set_current', 'helix_transmission_interfaces/SetCurrent')
req = roslibpy.ServiceRequest({"current" : -70.0})
result = srv.call(req)
print(result)

### Straighten the arm and update the calibration file

After moving the manipulator to the desired zero position, save the calibration file:

In [ ]:
# Save calibration file
srv = roslibpy.Service(client, '/tendon_transmission_node/set_motor_offsets', 'std_srvs/Trigger')
req = roslibpy.ServiceRequest()
result = srv.call(req)
print(result)

The controller can then be switched back to position control.

In [ ]:
# Switch back to position control
srv = roslibpy.Service(client, '/tendon_transmission_node/switch_to_position_control', 'std_srvs/Trigger')
req = roslibpy.ServiceRequest()
result = srv.call(req)
print(result)

### Checking the calibration after shutting down

When the motors are powered down, their absolute position might be lost. To detect and compensate for this when they are turned back on, the above steps should be followed up until 'Add tension to the arm', and then the 'check_calibration' service should be called, which will check for and correct the discrepancy. 

In [ ]:
# Check old calibration
srv = roslibpy.Service(client, '/tendon_transmission_node/check_calibration', 'std_srvs/Trigger')
req = roslibpy.ServiceRequest()
result = srv.call(req)
print(result)

# Switch back to position control
srv = roslibpy.Service(client, '/tendon_transmission_node/switch_to_position_control', 'std_srvs/Trigger')
req = roslibpy.ServiceRequest()
result = srv.call(req)
print(result)

## Tendon length control

### Simple example to read and command tendon lengths on the ROS topics

In [ ]:
# Subscribe to the tendon state information topic
listener = roslibpy.Topic(client, '/tendon_transmission_node/tendon_states', 'sensor_msgs/msg/JointState')
def listener_callback(message):
    print('Reading from /tendon_transmission_node/tendon_states')
    global joint_names 
    joint_names = message['name']
    global tendon_positions
    tendon_positions = message['position']
    # Stop listening after the first message
    listener.unsubscribe()

# Advertise to the command topic on the tendon transmission controller
talker = roslibpy.Topic(client, '/tendon_transmission_node/commands', 'std_msgs/msg/Float64MultiArray')

After setting up the subscriber and publisher, listen for a tendon state message

In [ ]:
listener.subscribe(listener_callback)

# Wait to receive message on /tendon_transmission_node/tendon_states
time.sleep(1)

# The listener callback has executed by now
print('Read joint names: ', joint_names)
print('Read tendon positions: ', tendon_positions)

Assuming the arm has just been calibrated, or the calibration has just been checked, all tendon states printed above should be close to 0 (within +/-0.03, ie 3cm).

After checking this, publish a command to put the arm in the zero position.

In [ ]:
talker.publish(roslibpy.Message({'data': [0,0,0,0,0,0,0,0,0]}))

And send other commands; the example below tightens the `joint8` tendon by 2cm, relative to the zero state.

In [ ]:
talker.publish(roslibpy.Message({'data': [0,0,0,0,0,0,0,0,-0.02]}))

### An example of interpolating a joint trajectory, instead of commanding a step input.

In [ ]:
# Create start position
zero = [0,0,0,0,0,0,0,0,0] 

# Create end position
j = [0,0,0,0,0,0,0,0,0]
j[1] = -0.02
j[4] = -0.03
j[7] = -0.04

# Create slowed down trajectory
T = 3 # time to move from zero to desired position (in seconds)
Update_freq = 50 # Update frequency of the commands (in Hertz). Note: Internally ROS writes to the motors at (max) 100Hz
Dxl_Positions = np.linspace(zero, j, num=T*Update_freq)

# Publish position commands
for ele in Dxl_Positions:
    talker.publish(roslibpy.Message({'data': list(ele)}))
    time.sleep(1/Update_freq)

# Cartesian Control

### Incremental control - joystick

If the SpaceMouse is connected to the Pi, calling the service below will turn on teleoperation of the end effector.

In [ ]:
srv = roslibpy.Service(client, '/helix_cartesian_control_node/activate_joystick_control', 'std_srvs/Trigger')
req = roslibpy.ServiceRequest()
result = srv.call(req)
print(result)

Now test moving the joystick around. The commands are sent in the 'origin' frame.

Turn off the joystick control, otherwise it will conflict with other Cartesian control.

In [ ]:
srv = roslibpy.Service(client, '/helix_cartesian_control_node/deactivate_joystick_control', 'std_srvs/Trigger')
req = roslibpy.ServiceRequest()
result = srv.call(req)
print(result)

### Incremental control - manual
You can also manually send incremental Cartesian control commands.

In [139]:
# Create the publisher and message
increment_pub = roslibpy.Topic(client, '/helix_cartesian_control_node/delta_increment', 'geometry_msgs/msgs/TwistStamped')
twist_msg = roslibpy.Message({
    "twist": {
        "linear": {"x": 0, "y": 0, "z": 0.002}, # Up 2mm
        "angular": {"x": 0.03, "y": 0, "z": 0}  # Rotate 0.03 rad around X axis
    }
})

In [152]:
# Publish the message (run multiple times to keep moving)
increment_pub.publish(twist_msg)

By default, the messages are sent in the 'origin' frame, but can be in any other valid frame.

In [133]:
twist_msg = roslibpy.Message({
    "header": {
        "frame_id":"arm_base"
    },
    "twist": {
        "linear": {"x": 0, "y": 0, "z": 0.002},
        "angular": {"x": -0.03, "y": 0, "z": 0}
    }
})

In [138]:
increment_pub.publish(twist_msg)

### Cartesian Pose to Pose Control

The below cells show how to call the services to move the end effector to a goal, using the different direction specifications, and options for goal frame and linear trajectroy planning.

In [73]:
### Reset the model and robot to calibrated zero position
srv = roslibpy.Service(client, '/helix_cartesian_control_node/reset_model', 'std_srvs/Trigger')
req = roslibpy.ServiceRequest()
result = srv.call(req)
print(result)

{'success': True, 'message': 'Model and robot reset to calibration state'}


In [14]:
### Move to a goal point with the gripper direction defined by a vector
srv = roslibpy.Service(client, '/helix_cartesian_control_node/go_to_gripper_pose_vector', 'helix_transmission_interfaces/GoToGripperPoseVector')
req = roslibpy.ServiceRequest({
  "goal_point":{"x": 0, "y": 0.25, "z": -0.4},
  "goal_direction":{"x": 0, "y": 1, "z": 0}
})
result = srv.call(req)
print(result)

{'success': True, 'message': 'Sucessfully planned to 1 of 1 path points.'}


In [12]:
### Move to a goal point with the gripper direction defined by a quaternion
srv = roslibpy.Service(client, '/helix_cartesian_control_node/go_to_gripper_pose_quat', 'helix_transmission_interfaces/GoToGripperPoseQuat')
req = roslibpy.ServiceRequest({
    "goal_pose":
  {
    "position":{"x": 0, "y": -0.25, "z": -0.4},
    "orientation":{"x": 0.7071, "y": 0, "z": 0, "w": 0.7071}
  }
})
result = srv.call(req)
print(result)

{'success': True, 'message': 'Sucessfully planned to 1 of 1 path points.'}


In [8]:
### Move to a goal point with the gripper direction defined by a set of Euler angles
srv = roslibpy.Service(client, '/helix_cartesian_control_node/go_to_gripper_pose_euler', 'helix_transmission_interfaces/GoToGripperPoseEuler')
req = roslibpy.ServiceRequest({
  "goal_point":{"x": 0, "y": 0.25, "z": -0.4},
  "goal_euler_angs":[-1.57, 0, 0],
  "axes":"sxyz"
})
result = srv.call(req)
print(result)

{'success': True, 'message': 'Sucessfully planned to 1 of 1 path points.'}


In [21]:
### Move to a goal in the 'arm_base' frame
srv = roslibpy.Service(client, '/helix_cartesian_control_node/go_to_gripper_pose_vector', 'helix_transmission_interfaces/GoToGripperPoseVector')
req = roslibpy.ServiceRequest({
  "frame_id":"arm_base",
  "goal_point":{"x": 0, "y": 0.25, "z": 0.4},
  "goal_direction":{"x": 0, "y": 1, "z": 0}
})
result = srv.call(req)
print(result)

{'success': True, 'message': 'Sucessfully planned to 1 of 1 path points.'}


In [22]:
### Move to a goal, planning a linear trajectory for the end effector
srv = roslibpy.Service(client, '/helix_cartesian_control_node/go_to_gripper_pose_vector', 'helix_transmission_interfaces/GoToGripperPoseVector')
req = roslibpy.ServiceRequest({
  "goal_point":{"x": 0, "y": 0, "z": -0.6},
  "goal_direction":{"x": 0, "y": 0, "z": -1},
  "plan_linear":True
})
result = srv.call(req)
print(result)

{'success': True, 'message': 'Sucessfully planned to 6 of 6 path points.'}


Terminate client after finishing the program

In [132]:
client.terminate()

AttributeError: 'TwistedEventLoopManager' object has no attribute '_thread'